In [ ]:
import pandas as pd

path = '/content/drive/MyDrive/TeamData/bank.csv'
df = pd.read_csv(path)
df.head()

,이탈여부,부양가족수,교육수준,총거래관계수,12개월비활성개월수,12개월고객접촉횟수,신용한도,회전잔액,1~4분기총이용금액변화,총거래금액,1~4분기거래횟수변화,나이그룹,수입,결혼여부_Married,결혼여부_Single,결혼여부_Unknown,카드등급_Gold,카드등급_Platinum,카드등급_Silver,성별_M
0,0.0,3,2,5,1,3,0.956476,-0.350175,1.335,-1.065558,1.625,2,3,True,False,False,False,False,False,True
1,0.0,5,4,6,1,2,0.435477,-0.289123,1.541,-1.008702,3.714,2,1,False,True,False,False,False,False,False
2,0.0,3,4,4,1,0,-0.132863,-0.895439,2.594,-0.778186,2.333,3,4,True,False,False,False,False,False,True
3,0.0,4,2,3,4,1,-0.145198,0.870877,1.405,-1.055115,2.333,1,1,False,False,True,False,False,False,False
4,0.0,3,1,5,1,0,0.019618,-0.895439,2.175,-1.192419,2.500,1,3,True,False,False,False,False,False,True


In [ ]:
print(df['이탈여부'].value_counts(normalize=True).round(3))
print(df['이탈여부'].value_counts())

이탈여부
0.0    0.839
1.0    0.161
Name: proportion, dtype: float64
이탈여부
0.0    8500
1.0    1627
Name: count, dtype: int64


In [ ]:
X = df.drop("이탈여부", axis=1)
y = df["이탈여부"]

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

def over_sampling(X,y):
    print(y.value_counts())
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    print(y_resampled.value_counts())
    print(X.shape, X_resampled.shape)

    return X_resampled, y_resampled

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    stratify=y,
    test_size=0.2,
    random_state=42
)
X_train, y_train = over_sampling(X_train,y_train)

이탈여부
0.0    6799
1.0    1302
Name: count, dtype: int64
이탈여부
0.0    6799
1.0    6799
Name: count, dtype: int64
(8101, 19) (13598, 19)


In [ ]:
# 데이터셋 크기 확인
print("데이터셋 분리 완료:")
print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Train label 분포:\n{y_train.value_counts(normalize=True).round(3)}")

데이터셋 분리 완료:
Train: (13598, 19), Test: (2026, 19)
Train label 분포:
이탈여부
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64


In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Ridge, Lasso
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import numpy as np

nomal_models = {
    'LogisticRegression': LogisticRegression(random_state=42), # 성지
    'KNN': KNeighborsClassifier(n_neighbors=5), # 용규
    "DecisionTree": DecisionTreeClassifier(random_state=42), # 석원
    "RandomForest": RandomForestClassifier(random_state=42), # 석원
    "XGBoost": XGBClassifier(random_state=42), # 성호
    "SVC": SVC(probability=True, random_state=42), # 용규
    "MLP": MLPClassifier(random_state=42, max_iter=1000), # 성지
}

nomal_voting_clf = VotingClassifier(
    estimators=[
        ('LogisticRegression', nomal_models["LogisticRegression"]),
        ('KNN', nomal_models["KNN"]),
        ('DecisionTree', nomal_models["DecisionTree"]),
        ('RandomForest', nomal_models["RandomForest"]),
        ('XGBoost', nomal_models["XGBoost"]),
        ('SVC', nomal_models["SVC"]),
        ('MLP', nomal_models["MLP"])
    ],
    voting='soft'  # 확률을 이용한 소프트 보팅 방식
)

nomal_models['VotingClassifier'] = nomal_voting_clf

# 딕셔너리 구조 초기화
nomal_model_scores = {}
for model_name in nomal_models.keys():
    nomal_model_scores[model_name] = [{}, {}]  # [0]: CV, [1]: Test

for model_name, model in nomal_models.items():
    f1 = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
    accuracy = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    precision = cross_val_score(model, X_train, y_train, cv=5, scoring='precision')
    roc_auc = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    recall = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')

    nomal_model_scores[model_name][0]['f1'] = np.mean(f1)
    nomal_model_scores[model_name][0]['accuracy'] = np.mean(accuracy)
    nomal_model_scores[model_name][0]['precision'] = np.mean(precision)
    nomal_model_scores[model_name][0]['roc_auc'] = np.mean(roc_auc)
    nomal_model_scores[model_name][0]['recall'] = np.mean(recall)

for model_name, model in nomal_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Lasso와 같은 회귀 모델의 경우 예측 값을 이진 분류로 변환
    if model_name in ['Ridge', 'Lasso']:  # 회귀 모델은 이진 분류로 변환 필요
        y_pred = (y_pred > 0.1).astype(int)

    # 평가
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    nomal_model_scores[model_name][1]['f1'] = f1
    nomal_model_scores[model_name][1]['accuracy'] = accuracy
    nomal_model_scores[model_name][1]['precision'] = precision
    nomal_model_scores[model_name][1]['roc_auc'] = roc_auc
    nomal_model_scores[model_name][1]['recall'] = recall


tuning_models = {
    'LogisticRegression': LogisticRegression(
        C=0.1,                 # 정규화 강도
        solver='liblinear',    # 최적화 알고리즘
        random_state=42
        ),
    'KNN': KNeighborsClassifier(
        n_neighbors=3,        # 예측 시 참조할 이웃 데이터 포인트 수
        weights='distance',   # 가까운 이웃일수록 더 큰 가중치 부여
        metric='cosine'       # 거리 측정 방식: 코사인 거리 사용 (각도 기반 유사도)
    ),
    "DecisionTree": DecisionTreeClassifier(
        criterion='entropy',        # 정보 이득 기준으로 분할 (기본은 'gini')
        max_depth=None,             # 트리 최대 깊이 제한 없음 (필요시 과적합 주의)
        max_features=5,             # 분할 시 고려할 최대 특성 수
        min_samples_split=3,        # 노드를 분할하기 위한 최소 샘플 수
        splitter='best',            # 최적의 분할 선택 (기본값)
        random_state=42             # 결과 재현을 위한 랜덤 시드
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=200,        # 트리 수 (기본값보다 많게)
        max_depth=10,            # 트리 최대 깊이 제한 → 과적합 방지
        min_samples_split=5,     # 내부 노드를 분할하는 데 필요한 최소 샘플 수
        min_samples_leaf=2,      # 리프 노드에 있어야 하는 최소 샘플 수
        max_features='sqrt',     # 각 노드에서 고려할 특성 수
        bootstrap=True,          # 배깅 방식
        random_state=42          # 재현성 보장
    ),
    "XGBoost": XGBClassifier(
        n_estimators=200,        # 트리 개수
        max_depth=5,             # 트리 최대 깊이 → 과적합 방지
        learning_rate=0.1,       # 학습률 (너무 작으면 학습 느림)
        subsample=0.8,           # 각 트리 학습 시 사용하는 샘플 비율
        colsample_bytree=0.8,    # 각 트리 생성 시 사용하는 feature 비율
        gamma=0,                 # 분할 최소 손실 감소 (클수록 보수적)
        reg_alpha=0.01,          # L1 정규화
        reg_lambda=1,            # L2 정규화
        use_label_encoder=False, # 경고 방지용
        eval_metric='logloss',   # 분류용 기본 평가 지표
        random_state=42          # 재현성
    ),
    "SVC": SVC(
        C=1,              # 오류 허용도 (작을수록 규제, 클수록 복잡한 모델 허용)
        gamma=0.1,        # RBF 커널의 감쇠 계수 (작을수록 멀리 있는 데이터도 영향)
        kernel="rbf",     # 비선형 관계에 강한 RBF 커널
        probability=True, # 소프트 보팅 등에서 확률 출력 필요
        random_state=42   # 결과 재현을 위한 랜덤 시드
    ),
    "MLP": MLPClassifier(
                    activation='relu',  # ReLU 활성화 함수 (입력값이 0보다 크면 그대로 반환, 0 이하일 때는 0 반환)
                    alpha=0.0001,  # L2 정규화 강도 (값이 클수록 과적합 방지)
                    batch_size='auto',  # 배치 크기를 시스템 메모리 기반으로 자동 설정
                    beta_1=0.9,  # Adam 옵티마이저의 1차 모멘텀 (일반적으로 0.9)
                    beta_2=0.999,  # Adam 옵티마이저의 2차 모멘텀
                    early_stopping=False,  # 조기 종료 사용 여부 (True로 설정하면 성능 향상이 멈추면 학습 종료)
                    epsilon=1e-08,  # Adam 옵티마이저의 수치적 안정성을 위한 작은 값
                    hidden_layer_sizes=(100,),  # 은닉층 노드 수 (하나의 은닉층에 100개의 뉴런)
                    learning_rate='constant',  # 학습률 일정하게 유지 ('constant')
                    learning_rate_init=0.001,  # 초기 학습률
                    max_fun=15000,  # 최대 함수 평가 횟수 (기본값: 15000)
                    max_iter=1000,  # 최대 반복 횟수 (1000번 학습)
                    momentum=0.9,  # 모멘텀 (이전 업데이트의 영향을 남겨서 더 빠르게 수렴)
                    n_iter_no_change=10,  # 10번 연속으로 성능 개선이 없으면 학습을 멈춤
                    nesterovs_momentum=True,  # Nesterov 모멘텀 사용 여부
                    power_t=0.5,  # 학습률 감소의 속도 조정
                    random_state=42,  # 랜덤 시드 (실험의 재현성 보장)
                    shuffle=True,  # 학습 전 데이터 섞기
                    solver='adam',  # 최적화 알고리즘 (Adam 사용)
                    tol=0.0001,  # 수렴 허용 오차 (손실 변화가 이 값보다 작으면 수렴했다고 판단)
                    validation_fraction=0.1,  # 검증 데이터 비율
                    verbose=False,  # 학습 로그 출력 여부
                    warm_start=False,  # 이전 학습 결과를 바탕으로 추가 학습을 할지 여부
                    ),  # 최적 파라미터 적용
}

tuning_voting_clf = VotingClassifier(
    estimators=[
        ('LogisticRegression', tuning_models["LogisticRegression"]),
        ('KNN', tuning_models["KNN"]),
        ('DecisionTree', tuning_models["DecisionTree"]),
        ('RandomForest', tuning_models["RandomForest"]),
        ('XGBoost', tuning_models["XGBoost"]),
        ('SVC', tuning_models["SVC"]),
        ('MLP', tuning_models["MLP"])
    ],
    voting='soft'  # 확률을 이용한 소프트 보팅 방식
)

# 튜닝 VotingClassifier도 추가
tuning_models['VotingClassifier'] = tuning_voting_clf

# tuning 모델에 대해 점수 저장용 딕셔너리 초기화
tuning_model_scores = {}
for model_name in tuning_models.keys():
    tuning_model_scores[model_name] = [{}, {}]  # [0]: CV, [1]: Test

# 각 튜닝 모델에 대해 교차검증 점수 측정
for model_name, model in tuning_models.items():
    f1 = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
    accuracy = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    precision = cross_val_score(model, X_train, y_train, cv=5, scoring='precision')
    roc_auc = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    recall = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')

    tuning_model_scores[model_name][0]['f1'] = np.mean(f1)
    tuning_model_scores[model_name][0]['accuracy'] = np.mean(accuracy)
    tuning_model_scores[model_name][0]['precision'] = np.mean(precision)
    tuning_model_scores[model_name][0]['roc_auc'] = np.mean(roc_auc)
    tuning_model_scores[model_name][0]['recall'] = np.mean(recall)

# 각 튜닝 모델에 대해 테스트 데이터로 평가
for model_name, model in tuning_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 평가 점수 계산
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    tuning_model_scores[model_name][1]['f1'] = f1
    tuning_model_scores[model_name][1]['accuracy'] = accuracy
    tuning_model_scores[model_name][1]['precision'] = precision
    tuning_model_scores[model_name][1]['roc_auc'] = roc_auc
    tuning_model_scores[model_name][1]['recall'] = recall

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:00:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:00:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:00:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:00:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:00:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

In [ ]:
import pandas as pd

def scores_to_wide_df(scores_dict):
    """
    scores_dict: {"모델이름": (cv_dict, test_dict)}
    """
    model_scores = {}
    for model_name, (cv_scores, test_scores) in scores_dict.items():
        row = {}
        # CV 점수 추가
        for metric, value in cv_scores.items():
            row[f"{metric}_cv"] = value
        # Test 점수 추가
        for metric, value in test_scores.items():
            row[f"{metric}_test"] = value
        model_scores[model_name] = row

    return pd.DataFrame(model_scores).T  # 모델별로 행으로 나누기

# 각각의 모델 점수 데이터프레임 생성
nomal_wide_df = scores_to_wide_df(nomal_model_scores)
tuning_wide_df = scores_to_wide_df(tuning_model_scores)

# 따로 저장
nomal_wide_df.to_csv("nomal_model_scores_wide_format.csv")
tuning_wide_df.to_csv("tuning_model_scores_wide_format.csv")
print(1)

In [13]:
!pip install -U numpy==1.26.4
!pip install -U autogluon

ERROR: Ignored the following yanked versions: 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15
ERROR: Ignored the following versions that require a different python version: 0.1.0 Requires-Python >=3.6, <3.9; 0.1.0b20210207 Requires-Python >=3.6, <3.8; 0.1.0b20210208 Requires-Python >=3.6, <3.8; 0.1.0b20210209 Requires-Python >=3.6, <3.8; 0.1.0b20210210 Requires-Python >=3.6, <3.8; 0.1.0b20210211 Requires-Python >=3.6, <3.8; 0.1.0b20210212 Requires-Python >=3.6, <3.8; 0.1.0b20210213 Requires-Python >=3.6, <3.8; 0.1.0b20210214 Requires-Python >=3.6, <3.8; 0.1.0b20210215 Requires-Python >=3.6, <3.8; 0.1.0b20210216 Requires-Python >=3.6, <3.8; 0.1.0b20210217 Requires-Python >=3.6, <3.8; 0.1.0b20210218 Requires-Python >=3.6, <3.8; 0.1.0b20210219 Requires-Python >=3.6, <3.8; 0.1.0b20210220 Requires-Python >=3.6, <3.8; 0.1.0b20210221 Requires-Python >=3.6, <3.8; 0.1.0b20210222 Requires-Python >=3.6, <3.8; 0.1.0b20210223 Requires-Python >=

In [9]:
import pandas as pd

path = '/content/drive/MyDrive/TeamData/bank.csv'
df = pd.read_csv(path)
df.head()

,이탈여부,부양가족수,교육수준,총거래관계수,12개월비활성개월수,12개월고객접촉횟수,신용한도,회전잔액,1~4분기총이용금액변화,총거래금액,1~4분기거래횟수변화,나이그룹,수입,결혼여부_Married,결혼여부_Single,결혼여부_Unknown,카드등급_Gold,카드등급_Platinum,카드등급_Silver,성별_M
0,0.0,3,2,5,1,3,0.956476,-0.350175,1.335,-1.065558,1.625,2,3,True,False,False,False,False,False,True
1,0.0,5,4,6,1,2,0.435477,-0.289123,1.541,-1.008702,3.714,2,1,False,True,False,False,False,False,False
2,0.0,3,4,4,1,0,-0.132863,-0.895439,2.594,-0.778186,2.333,3,4,True,False,False,False,False,False,True
3,0.0,4,2,3,4,1,-0.145198,0.870877,1.405,-1.055115,2.333,1,1,False,False,True,False,False,False,False
4,0.0,3,1,5,1,0,0.019618,-0.895439,2.175,-1.192419,2.500,1,3,True,False,False,False,False,False,True


In [10]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

def makeDf(X,y):
  df = pd.DataFrame(X)
  df['target'] = y
  return df

def over_sampling(X,y):
    print(y.value_counts())
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    print(y_resampled.value_counts())
    print(X.shape, X_resampled.shape)

    return X_resampled, y_resampled
X = df.drop("이탈여부", axis=1)
y = df["이탈여부"]

df_123 = makeDf(X,y)

df_train, df_test = train_test_split(df_123, random_state=42)

a,b= over_sampling(df_train.drop('target',axis=1), df_train.loc[:,'target'] )
df_train = makeDf(a,b )

# TabularDataset
bank_train = TabularDataset(df_train)
bank_test = TabularDataset(df_test)

target
0.0    6387
1.0    1208
Name: count, dtype: int64
target
0.0    6387
1.0    6387
Name: count, dtype: int64
(7595, 19) (12774, 19)


In [18]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score

# --- 모델 불러오기 ---
model_path = '/content/drive/MyDrive/Bank_model_0417'
predictor = TabularPredictor.load(model_path)

# --- 예측 ---
y_pred = predictor.predict(bank_test)
y_pred_proba = predictor.predict_proba(bank_test).iloc[:, 1]  # 클래스 1 확률만

# --- 교차 검증 (CV 점수) 계산 ---
# AutoGluon에서 교차 검증을 직접 하려면 데이터를 다시 적합시키고 evaluate 메서드를 사용해야 합니다.
cv_scores = predictor.evaluate(bank_train)  # train 데이터에 대해 평가

# --- CV 점수 정리 ---
cv_metrics = {
    "f1": cv_scores["f1"],
    "accuracy": cv_scores["accuracy"],
    "precision": cv_scores["precision"],
    "roc_auc": cv_scores["roc_auc"],
    "recall": cv_scores["recall"],
}

# --- Test 성능 계산 (순서: f1 → accuracy → precision → roc_auc → recall) ---
test_metrics = {
    "f1": f1_score(bank_test['target'], y_pred),
    "accuracy": accuracy_score(bank_test['target'], y_pred),
    "precision": precision_score(bank_test['target'], y_pred),
    "roc_auc": roc_auc_score(bank_test['target'], y_pred_proba),
    "recall": recall_score(bank_test['target'], y_pred),
}

# --- 결과 통합 저장 ---
automl_model_scores = {
    "AutoML": (cv_metrics, test_metrics)
}

# --- wide 포맷 변환 함수 ---
def scores_to_wide_df(scores_dict):
    model_scores = {}
    for model_name, (cv_scores, test_scores) in scores_dict.items():
        row = {}
        for metric, value in cv_scores.items():
            row[f"{metric}_cv"] = value
        for metric, value in test_scores.items():
            row[f"{metric}_test"] = value
        model_scores[model_name] = row
    return pd.DataFrame(model_scores).T

# --- 데이터프레임 생성 및 저장 ---
automl_wide_df = scores_to_wide_df(automl_model_scores)
automl_wide_df.to_csv("automl_model_scores_wide_format.csv")

print("✅ 저장 완료: automl_model_scores_wide_format.csv")


✅ 저장 완료: automl_model_scores_wide_format.csv
